In [1]:
# workspace/notebooks/gpu/quick_test.ipynb

import torch
import sys
import os

# Set ROCm environment
os.environ['MIOPEN_DISABLE_CACHE'] = '1'

print("="*70)
print("Quick ROCm Test")
print("="*70)

print(f"\nPyTorch version: {torch.__version__}")
print(f"ROCm available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Test 1: Import ROCm ResNet
    print(f"\n1. Testing ROCm ResNet import...")
    try:
        sys.path.append('/workspace/notebooks/gpu')
        from rocm_resnet import rocm_resnet18
        
        model = rocm_resnet18().to(device)
        print(f"✓ ROCm ResNet18 created: {sum(p.numel() for p in model.parameters()):,} params")
        
        # Test forward
        x = torch.randn(64, 3, 32, 32, device=device)
        with torch.no_grad():
            y = model(x)
        print(f"✓ Forward pass: {x.shape} -> {y.shape}")
        
    except Exception as e:
        print(f"✗ Import failed: {e}")
    
    # Test 2: Large memory allocation
    print(f"\n2. Testing 68GB VRAM...")
    try:
        # Try allocating 30GB tensor
        tensor = torch.randn(10000, 10000, device=device)  # ~400MB
        print(f"✓ Allocated {tensor.element_size() * tensor.nelement() / 1e9:.2f} GB")
        
        # Try larger
        tensor_large = torch.randn(20000, 20000, device=device)  # ~1.6GB
        print(f"✓ Allocated {tensor_large.element_size() * tensor_large.nelement() / 1e9:.2f} GB")
        
        # Test operation
        result = tensor_large @ tensor_large.T
        print(f"✓ Matrix multiplication successful")
        print(f"  Memory used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        
    except Exception as e:
        print(f"✗ Memory test failed: {type(e).__name__}")
    
    print(f"\n" + "="*70)
    print(f"✅ Your 68GB VRAM ROCm setup is working!")
    print(f"="*70)
    
else:
    print("✗ No GPU available!")

Quick ROCm Test

PyTorch version: 2.9.1+rocm7.1.1.git351ff442
ROCm available: True
Device: AMD Radeon Graphics
VRAM: 68.72 GB

1. Testing ROCm ResNet import...
✓ ROCm ResNet18 created: 11,173,962 params
✓ Forward pass: torch.Size([64, 3, 32, 32]) -> torch.Size([64, 10])

2. Testing 68GB VRAM...
✓ Allocated 0.40 GB
✓ Allocated 1.60 GB
✓ Matrix multiplication successful
  Memory used: 3.73 GB

✅ Your 68GB VRAM ROCm setup is working!
